In [1]:
#%% 
import pandas as pd
import spacy
from nltk.sentiment import SentimentIntensityAnalyzer
import sqlite3
# from bertopic import BERTopic


ModuleNotFoundError: No module named 'spacy'

In [ ]:
#%% 
import pandas as pd
import spacy
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.util import ngrams
from nltk.corpus import stopwords
import nltk

# Download stopwords from nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the CSV file
file_path = 'climatenews_subreddit.csv'  # Replace with your actual file path
data = pd.read_csv(file_path)

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to calculate VADER sentiment score
def get_vader_sentiment(text):
    return sia.polarity_scores(text)['compound']

# Function to generate and filter n-grams
def generate_filtered_ngrams(text, n=2):
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    generated_ngrams = ngrams(filtered_words, n)
    return ', '.join([' '.join(gram) for gram in generated_ngrams])

# Create a new DataFrame for the co-occurrence table
co_occurrence = pd.DataFrame()

# Fill the new DataFrame with the required data
co_occurrence['user'] = data['user']
co_occurrence['author'] = data['author']
co_occurrence['comment'] = data['comment']
co_occurrence['title'] = data['title']
co_occurrence['post_text'] = data['post_text']
co_occurrence['comm_date'] = data['comm_date']
co_occurrence['subreddit'] = data['url'].apply(lambda url: url.split('/')[4] if len(url.split('/')) > 4 else 'Unknown')

# Adding N-grams and sentiment analysis results
co_occurrence['ngrams.title'] = co_occurrence['title'].astype(str).apply(lambda x: generate_filtered_ngrams(x, 2))
co_occurrence['ngrams.comment'] = co_occurrence['comment'].astype(str).apply(lambda x: generate_filtered_ngrams(x, 2))
co_occurrence['vader.title'] = co_occurrence['title'].astype(str).apply(get_vader_sentiment)
co_occurrence['vader.comment'] = co_occurrence['comment'].astype(str).apply(get_vader_sentiment)

# Handle missing values if necessary (e.g., with empty strings)
co_occurrence.fillna('', inplace=True)

# Save the co-occurrence table with additional data to a new CSV file
output_file_path = 'enhanced_co_occurrence_table_with_ngrams.csv'  # Replace with your desired output file path
co_occurrence.to_csv(output_file_path, index=False)

print("Enhanced co-occurrence table with N-grams created and saved to:", output_file_path)
